# Load packages

In [1]:
import os
import gc
import numpy as np
import pandas as pd
from glob import glob
from joblib import delayed, Parallel
import matplotlib.pyplot as plt
# import dask.dataframe as dd
# import dask.array as da
%matplotlib inline
pd.set_option('display.max_columns', 500)

In [2]:
ts_files = sorted(glob('./data/training_set/*.csv.gz'))

# Check if the training data files are sorted

In [4]:
def check_if_file_is_sorted(x):
    ts = pd.read_csv(x)
    original_index = ts.index.values.copy()
    ts.sort_values(['session_id', 'session_position'], inplace=True)
    new_index = ts.index.values.copy()
    is_sorted = np.all(original_index == new_index)
    ts['ts'] = pd.to_datetime(ts.date + ' ' + ts.hour_of_day.astype('str') + ':00:00')
    min_ts = ts.ts.min()
    max_ts = ts.ts.max()
    is_ts_sorted = np.all(ts.ts.values == ts.ts.sort_values().values)
    return (x, is_sorted, min_ts, max_ts, is_ts_sorted)

In [5]:
tmp = Parallel(14)(delayed(check_if_file_is_sorted)(x) for x in ts_files)

In [6]:
len(tmp)

660

In [7]:
set(str(type(x)) for x in tmp)

{"<class 'tuple'>"}

In [8]:
tmp[0]

('./data/training_set/log_0_20180715_000000000000.csv.gz',
 True,
 Timestamp('2015-09-30 21:00:00'),
 Timestamp('2018-08-22 11:00:00'),
 False)

In [9]:
df = pd.DataFrame.from_records(tmp, columns=[
    'filepath', 'is_sorted', 'min_ts', 'max_ts', 'is_ts_sorted'])

In [10]:
df.is_sorted.all()

True

In [11]:
df.min_ts.min()

Timestamp('1969-12-31 16:00:00')

In [12]:
df.max_ts.max()

Timestamp('2032-05-26 00:00:00')

In [13]:
df.sample(2)

,filepath,is_sorted,min_ts,max_ts,is_ts_sorted
370,./data/training_set/log_5_20180824_00000000000...,True,2017-07-01 21:00:00,2018-10-08 06:00:00,False
420,./data/training_set/log_6_20180808_00000000000...,True,1970-01-01 20:00:00,2018-08-09 12:00:00,False


In [14]:
del df
del tmp

# Check which columns are (nearly) unique per session

In [15]:
ts = pd.read_csv(ts_files[0], dtype='str')

In [16]:
ts.head()

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,short_pause_before_play,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,1,20,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,false,false,false,true,0,0,0,0,0,0,true,16,2018-07-15,true,editorial_playlist,trackdone,trackdone
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,2,20,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,false,false,false,true,0,1,0,0,0,0,true,16,2018-07-15,true,editorial_playlist,trackdone,trackdone
2,0_00006f66-33e5-4de7-a324-2d18e439fc1e,3,20,t_fc5df5ba-5396-49a7-8b29-35d0d28249e0,false,false,false,true,0,1,0,0,0,0,true,16,2018-07-15,true,editorial_playlist,trackdone,trackdone
3,0_00006f66-33e5-4de7-a324-2d18e439fc1e,4,20,t_23cff8d6-d874-4b20-83dc-94e450e8aa20,false,false,false,true,0,1,0,0,0,0,true,16,2018-07-15,true,editorial_playlist,trackdone,trackdone
4,0_00006f66-33e5-4de7-a324-2d18e439fc1e,5,20,t_64f3743c-f624-46bb-a579-0f3f9a07a123,false,false,false,true,0,1,0,0,0,0,true,16,2018-07-15,true,editorial_playlist,trackdone,trackdone


In [17]:
ts.skip_2.value_counts()

true     1533436
false    1457173
Name: skip_2, dtype: int64

In [18]:
ts.not_skipped.value_counts()

false    1985901
true     1004708
Name: not_skipped, dtype: int64

In [19]:
((ts.skip_2 == 'true') == (ts.not_skipped == 'false')).all()

False

In [20]:
len(ts.session_id.unique())

178342

In [21]:
ts.loc[:, ['session_id', 'hist_user_behavior_reason_end']].drop_duplicates().shape

(521578, 2)

In [22]:
ts.loc[:, ['session_id', 'hist_user_behavior_reason_start']].drop_duplicates().shape

(554160, 2)

In [23]:
ts.loc[:, ['session_id', 'context_type']].drop_duplicates().shape

(230742, 2)

In [24]:
ts.loc[:, ['session_id', 'premium']].drop_duplicates().shape

(178366, 2)

In [25]:
ts.loc[:, ['session_id', 'date']].drop_duplicates().shape

(181173, 2)

In [26]:
ts.loc[:, ['session_id', 'hist_user_behavior_is_shuffle']].drop_duplicates().shape

(200598, 2)

In [27]:
ts.loc[:, ['session_id', 'hist_user_behavior_n_seekback']].drop_duplicates().shape

(240121, 2)

In [28]:
ts.loc[:, ['session_id', 'hist_user_behavior_n_seekfwd']].drop_duplicates().shape

(232455, 2)

In [29]:
ts.loc[:, ['session_id', 'long_pause_before_play']].drop_duplicates().shape

(336010, 2)

In [30]:
ts.loc[:, ['session_id', 'short_pause_before_play']].drop_duplicates().shape

(325376, 2)

In [31]:
ts.loc[:, ['session_id', 'no_pause_before_play']].drop_duplicates().shape

(356655, 2)

In [32]:
ts.loc[:, ['session_id', 'context_switch']].drop_duplicates().shape

(241712, 2)

# Extract session related information

In [33]:
gc.collect()

1161

In [34]:
def uniq_session_ids(x):
    ts = pd.read_csv(x, dtype='str')
    ts = ts.groupby('session_id')['session_length', 'date', 'premium'].first().reset_index().assign(filepath = x)
    return ts

In [35]:
tmp = Parallel(14)(delayed(uniq_session_ids)(x) for x in ts_files)

In [36]:
df = pd.concat(tmp)
df.shape

(124950714, 5)

In [37]:
df.to_pickle('./data/all_session_ids.pkl')

# Distribution of session_length, year and premium

In [38]:
df = pd.read_pickle('./data/all_session_ids.pkl.gz')
df.shape

(124950714, 5)

In [39]:
len(df.session_id.unique())

124950714

In [40]:
df.head()

,session_id,session_length,date,premium,filepath
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,20,2018-07-15,true,./data/training_set/log_0_20180715_00000000000...
1,0_0000a72b-09ac-412f-b452-9b9e79bded8f,20,2018-07-15,true,./data/training_set/log_0_20180715_00000000000...
2,0_00010fc5-b79e-4cdf-bc4c-f140d0f99a3a,20,2018-07-15,true,./data/training_set/log_0_20180715_00000000000...
3,0_00016a3d-9076-4f67-918f-f29e3ce160dc,20,2018-07-15,true,./data/training_set/log_0_20180715_00000000000...
4,0_00018b58-deb8-4f98-ac5e-d7e01b346130,11,2018-07-15,true,./data/training_set/log_0_20180715_00000000000...


In [41]:
df.session_length = df.session_length.astype('int64')

In [42]:
df.session_length.value_counts().sort_index()

10    11025730
11     9611839
12     8407692
13     7344162
14     6460309
15     5664259
16     4994670
17     4400713
18     3917839
19     3477536
20    59645965
Name: session_length, dtype: int64

In [43]:
df.session_length.value_counts(normalize=True, sort=False).sort_index()*100

10     8.824063
11     7.692504
12     6.728807
13     5.877647
14     5.170286
15     4.533195
16     3.997312
17     3.521959
18     3.135507
19     2.783126
20    47.735594
Name: session_length, dtype: float64

In [44]:
df.premium.value_counts(normalize=True)

true     0.816271
false    0.183729
Name: premium, dtype: float64

In [45]:
(df.date.str[:4].value_counts(normalize=True, sort=False).sort_index() * 100)\
    .apply(lambda x: '%.7f' % x)

1969     0.0001104
1970     0.0001409
1972     0.0000008
1999     0.0000048
2000     0.0000104
2008     0.0000016
2009     0.0000200
2010     0.0000512
2011     0.0000160
2012     0.0000112
2013     0.0000216
2014     0.0000448
2015     0.0001785
2016     0.0002105
2017     0.0011100
2018    99.9980480
2019     0.0000104
2031     0.0000008
2032     0.0000080
Name: date, dtype: object